# Biodata
* Nama: **Virgiawan Teguh Kusuma**
* Email: **181240000833@unisnu.ac.id**
* Domisili: **Kabupaten Jepara, Jawa Tengah**
* Dataset: https://www.kaggle.com/shreyaspj/android-devices-and-mobiles

# Load Library

## Load google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load Library lainnya

In [2]:
import pandas as pd
import zipfile, os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Load Dataset

In [3]:
phone = '/content/drive/MyDrive/archive_7.zip'
phoneRef = zipfile.ZipFile(phone, 'r')
phoneRef.extractall('/content')
phoneRef.close()

In [4]:
dataframe = pd.read_csv('mobiles1.csv')
# dataframe

In [5]:
dataframe = dataframe.drop(columns=['camera',
                                    'rating',
                                    'reviews',
                                    'warranty'])
# dataframe

In [6]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   battery    984 non-null    object
 1   display    984 non-null    object
 2   memory     984 non-null    object
 3   name       984 non-null    object
 4   price      984 non-null    int64 
 5   processor  983 non-null    object
dtypes: int64(1), object(5)
memory usage: 46.2+ KB


In [7]:
data_battery= dataframe['battery'].str.split('(\d+)',  expand=True)

#Split kolom memori menjadi atribut memori dan tipe penyimpanan, biar mudah dalam melihat data
split_memori = dataframe['memory'].str.split('|', 1, expand=True)
split_display= dataframe['display'].str.split('(', 1, expand=True)
# split_processor= dataframe['processor'].str.split('GHz', 0, expand=True)
# split_memori
# split_display
#Inisiasi kolom baru yaitu tipe penyimpanan

dataframe['battery(mAh)'] = pd.to_numeric(data_battery[1], downcast='float')

dataframe['display(inch)'] = split_display[1]
data_display= dataframe['display(inch)'].str.split('(\d+)',  expand=True)

dataframe['display(inch)'] = pd.to_numeric(data_display[1], downcast='float')

# dataframe['processor(GHz)'] = split_processor[0]
dataframe['RAM(GB)'] = split_memori[0]
dataframe['ROM(GB)'] = split_memori[1]

dataframe = dataframe.drop(columns=['battery',
                                    'memory',
                                    'display',
                                    'processor'])
dataframe

,name,price,battery(mAh),display(inch),RAM(GB),ROM(GB)
0,"Redmi 8 (Ruby Red, 64 GB)",9999,5000.0,6.0,4 GB RAM,64 GB ROM | Expandable Upto 512 GB
1,"Realme 5i (Aqua Blue, 64 GB)",10999,5000.0,6.0,4 GB RAM,64 GB ROM
2,"Realme 5i (Aqua Blue, 128 GB)",11999,5000.0,6.0,4 GB RAM,128 GB ROM
3,"Realme 5i (Forest Green, 128 GB)",11999,5000.0,6.0,4 GB RAM,128 GB ROM
4,"Realme C2 (Diamond Blue, 32 GB)",7499,4000.0,6.0,3 GB RAM,32 GB ROM | Expandable Upto 256 GB
...,...,...,...,...,...,...
979,"Micromax Bharat 4 (Black, 8 GB)",3590,2000.0,5.0,1 GB RAM,8 GB ROM | Expandable Upto 32 MB
980,"Nextbit Robin (Ember, 32 GB)",19999,2680.0,5.0,3 GB RAM,32 GB ROM
981,"Gionee A1 Plus (Mocha Gold, 64 GB)",10499,4550.0,6.0,4 GB RAM,64 GB ROM | Expandable Upto 256 GB
982,"XOLO Omega 5.0 (Black, 8 GB)",8990,2100.0,5.0,1 GB RAM,8 GB ROM | Expandable Upto 32 GB


In [8]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           984 non-null    object 
 1   price          984 non-null    int64  
 2   battery(mAh)   984 non-null    float32
 3   display(inch)  984 non-null    float32
 4   RAM(GB)        984 non-null    object 
 5   ROM(GB)        984 non-null    object 
dtypes: float32(2), int64(1), object(3)
memory usage: 38.6+ KB


In [9]:
data_ram= dataframe['RAM(GB)'].str.split('(\d+)',  expand=True)
data_rom= dataframe['ROM(GB)'].str.split('(\d+)',  expand=True)
# data_processor= dataframe['processor(GHz)'].str.split('(\d+)',  expand=True)
dataframe['RAM(GB)'] = pd.to_numeric(data_ram[1], downcast='integer')
dataframe['ROM(GB)'] = pd.to_numeric(data_rom[1], downcast='integer')
# dataframe['processor(GHz)'] = pd.to_numeric(data_processor[1], downcast='float')
dataframe['price(IDR)'] = (dataframe['price'] * 191.28)
dataframe = dataframe.drop(columns=['price', 'name'])
dataframe.head()

,battery(mAh),display(inch),RAM(GB),ROM(GB),price(IDR)
0,5000.0,6.0,4,64,1912608.72
1,5000.0,6.0,4,64,2103888.72
2,5000.0,6.0,4,128,2295168.72
3,5000.0,6.0,4,128,2295168.72
4,4000.0,6.0,3,32,1434408.72


In [10]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery(mAh)   984 non-null    float32
 1   display(inch)  984 non-null    float32
 2   RAM(GB)        984 non-null    int16  
 3   ROM(GB)        984 non-null    int16  
 4   price(IDR)     984 non-null    float64
dtypes: float32(2), float64(1), int16(2)
memory usage: 19.3 KB


In [11]:
loc_data=dataframe.loc[
                       (dataframe['display(inch)'] >= 5.8) &
                       (dataframe['battery(mAh)'] >= 3000) &
                       (dataframe['RAM(GB)'] >= 3) &
                       (dataframe['ROM(GB)'] >= 32)
                       ]

In [12]:
newDataframe = loc_data
newDataframe.head

<bound method NDFrame.head of      battery(mAh)  display(inch)  RAM(GB)  ROM(GB)  price(IDR)
0          5000.0            6.0        4       64  1912608.72
1          5000.0            6.0        4       64  2103888.72
2          5000.0            6.0        4      128  2295168.72
3          5000.0            6.0        4      128  2295168.72
4          4000.0            6.0        3       32  1434408.72
..            ...            ...      ...      ...         ...
973        3700.0            6.0        4       64  6694608.72
974        4000.0            6.0        6       64  4014967.20
975        4000.0            6.0        6       64  4780087.20
976        5000.0            6.0        6       64  2677728.72
981        4550.0            6.0        4       64  2008248.72

[615 rows x 5 columns]>

In [13]:
newDataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 615 entries, 0 to 981
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery(mAh)   615 non-null    float32
 1   display(inch)  615 non-null    float32
 2   RAM(GB)        615 non-null    int16  
 3   ROM(GB)        615 non-null    int16  
 4   price(IDR)     615 non-null    float64
dtypes: float32(2), float64(1), int16(2)
memory usage: 16.8 KB


In [14]:
newDataframe.head()

,battery(mAh),display(inch),RAM(GB),ROM(GB),price(IDR)
0,5000.0,6.0,4,64,1912608.72
1,5000.0,6.0,4,64,2103888.72
2,5000.0,6.0,4,128,2295168.72
3,5000.0,6.0,4,128,2295168.72
4,4000.0,6.0,3,32,1434408.72


In [15]:
newDataframe.shape

(615, 5)

In [16]:
newDataframe.describe()

,battery(mAh),display(inch),RAM(GB),ROM(GB),price(IDR)
count,615.000000,615.000000,615.00000,615.000000,6.150000e+02
mean,4293.569336,6.003252,5.43252,103.284553,3.782323e+06
std,583.830383,0.056980,2.03109,67.896132,2.592351e+06
min,3000.000000,6.000000,3.00000,32.000000,1.090105e+06
25%,4000.000000,6.000000,4.00000,64.000000,2.198668e+06
50%,4035.000000,6.000000,6.00000,64.000000,3.136801e+06
75%,5000.000000,6.000000,6.00000,128.000000,4.207969e+06
max,6000.000000,7.000000,12.00000,512.000000,2.008421e+07


cek missing value

In [17]:
newDataframe.isnull().sum()

battery(mAh)     0
display(inch)    0
RAM(GB)          0
ROM(GB)          0
price(IDR)       0
dtype: int64

menangani missing value

In [18]:
# newDataframe.dropna(subset=['processor'], inplace=True)
# newDataframe.isnull().sum()

# Data Modeling

In [19]:
newDataframe = newDataframe.drop(columns=['display(inch)'])

#Pertama, buat variabel x dan y.
x = newDataframe.drop(columns='price(IDR)')
y = newDataframe['price(IDR)']
#Kedua, ucup split data menjadi training and testing dengan porsi 80:20.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=4)
#Ketiga, ucup bikin object linear regresi.
lin_reg = LinearRegression()
#Keempat, train the model menggunakan training data yang sudah displit.
lin_reg.fit(x_train, y_train)
#Kelima, cari tau nilai slope/koefisien (m) dan intercept (b).
print(lin_reg.coef_)
print(lin_reg.intercept_)

[-3.6708965e+01  8.1079825e+05  5.8328750e+03]
-1054189.8


In [20]:
lin_reg.score(x_test, y_test)

0.6767437125339462

In [21]:
newDataframe.head()

,battery(mAh),RAM(GB),ROM(GB),price(IDR)
0,5000.0,4,64,1912608.72
1,5000.0,4,64,2103888.72
2,5000.0,4,128,2295168.72
3,5000.0,4,128,2295168.72
4,4000.0,3,32,1434408.72


In [26]:
price = lin_reg.predict([[3000,4,64]])
price = np.round([price], decimals=1)

In [27]:
print('perkiraan harga handphone adalah Rp.',price[0][0])

perkiraan harga handphone adalah Rp. 2452180.4
